# Train your first Deep Reinforcement Learning Agent

PYTHON 3.9

El objetivo de este notebook es crear un sistema para entrenar los algoritmos que sea claro y escalable. Así que voy a tratar el menor número de celdas posibles

In [1]:
import sys #-- lo he necesitado para importar el paquete
sys.path.append('..')
import gym_hearthstone

## Argumentos
He añadido argumentos al environment para que sea más sencillo hacer el entrenamiento:

    action_type: Define el espacio de acción que se va a crear. Ahora mismo solo hay tres implementados.
    
        * "random"  -> Solo una acción possible (Tomar una acción aleatoria de las que son posibles).
        * "type_rd" -> 6 acciones posibles (Escoge un tipo de accion sino es legal hace una de cualquier tipo).
        * "type"    -> 6 acciones posibles (Toma una aleatoria según el tipo de acción y sino es legal no hace nada).
        
        
        
    reward_mode: Define el reward que devuelve el environment:
    
        * "simple"    -> Solo hay reward cuando se gana o se pierde.
        * "penalize"  -> Se penalizan las acciones que no son válidas.
        * "incentive" -> Se premian las acciones que son válidas.
        * "complex"   -> Se penalizan las acciones que no son válidas y se premian las válidas.


    steps: Por ahora usa el numero de steps para el nombre del fichero de salida. 
    
        

In [2]:
import gymnasium as gym
import gym_hearthstone
from stable_baselines3 import DQN
import time
import os

In [3]:

steps = 1000
run = -1
env = gym.make('Hearthstone-v1', action_type = "type", reward_mode = "complex")

[fireplace.__init__]: Initializing card database
[fireplace.__init__]: Merged 26660 cards
[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player2 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ("Gul'dan")>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Life Tap')>)> targeting [Player(name='Player1', hero=<Hero ("Gul'dan")>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Life Tap')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ("Gul'dan")>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ("Gul'dan")>)> targeting [Player(name='Player1', hero=<Hero ("Gul'dan")>)]
[fireplace.actions]: Player1 summons [<Hero ("Gul'dan")>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ("Gul'dan")

IMPLEMENTED CARDS: 8047
UNIMPLEMENTED CARDS: 18613
Implemented but Uncollectible: 6145
CardClass.DEATHKNIGHT amount of cards is: 0
CardClass.DRUID amount of cards is: 130
CardClass.HUNTER amount of cards is: 132
CardClass.MAGE amount of cards is: 128
CardClass.PALADIN amount of cards is: 128
CardClass.PRIEST amount of cards is: 119
CardClass.ROGUE amount of cards is: 123
CardClass.SHAMAN amount of cards is: 126
CardClass.WARLOCK amount of cards is: 118
CardClass.WARRIOR amount of cards is: 120
CardClass.DEMONHUNTER amount of cards is: 41
CardClass.NEUTRAL amount of cards is: 737
HearthstoneEnv - Version 0.2.0
Step.BEGIN_MULLIGAN
Step.MAIN_ACTION


In [4]:
run += 1
## Generar Modelo
model = DQN("MultiInputPolicy", env, verbose=1)

# Resetear env y stats
env.reset_stats()
start = time.time()
#env.reset()

## Entrenar modelo
model.learn(total_timesteps=steps, log_interval=4)
end = time.time()




Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\jesus\anaconda3\envs\env_gym-hearthstone\lib\site-packages\stable_baselines3\common\buffers.py:564: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 5.84GB > 0.54GB
  warnings.warn(
C:\Users\jesus\anaconda3\envs\env_gym-hearthstone\lib\site-packages\gymnasium\utils\passive_env_checker.py:197: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
C:\Users\jesus\anaconda3\envs\env_gym-hearthstone\lib\site-packages\gymnasium\utils\passive_env_checker.py:210: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[firep

Step.BEGIN_MULLIGAN
Step.MAIN_ACTION
1633
1145
1992
3144
4907
2848
3619
1423
7943
------------------------
>>> Current Step 1
------------------------

>>> possible_actions 3:[[<Move.choice: 5>, 0], [<Move.choice: 5>, 1], [<Move.choice: 5>, 2]]
>>> alreadySelectedActions 0:[]
>>> possible_actions 3:[[<Move.choice: 5>, 0], [<Move.choice: 5>, 1], [<Move.choice: 5>, 2]]
doing single action for AI[<Move.choice: 5>, 2]
3661
1526
1633
1992
3144
4907
2848
3619
1423
7943
------------------------
>>> Current Step 2
------------------------
{'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 3, None, None]], 'heropower': [[<Move.hero_power: 1>, None, None]], 'minion_attack': [], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}

>>> possible_actions 5:[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 3, None, None], [<Move.hero_power: 1>, None, None], [<Move.end_turn: 

[fireplace.actions]: Player2 plays <Minion ('Abusive Sergeant')> (target=None, index=None)
[fireplace.entity]: Player2 pays 1 mana
[fireplace.actions]: %r requires a target for its battlecry. Will not trigger.
[fireplace.card]: <Minion ('Abusive Sergeant')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


------------------------
>>> Current Step 5
------------------------
{'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 3, None, None]], 'heropower': [[<Move.hero_power: 1>, None, None]], 'minion_attack': [], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}

>>> possible_actions 5:[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 3, None, None], [<Move.hero_power: 1>, None, None], [<Move.end_turn: 0>]]
>>> alreadySelectedActions 0:[]
>>> possible_actions 5:[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 3, None, None], [<Move.hero_power: 1>, None, None], [<Move.end_turn: 0>]]
3661
1526
1633
1992
3144
4907
2848
3619
1423
7943
------------------------
>>> Current Step 6
------------------------
{'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play

...

{'choice': [], 'play_card': [[<Move.play_card: 4>, 1, None, None]], 'heropower': [], 'minion_attack': [], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}
>>> possible_actions 2:[[<Move.play_card: 4>, 1, None, None], [<Move.end_turn: 0>]]
>>> alreadySelectedActions 0:[]

doing end turn for rando
Doing action: [<Move.end_turn: 0>]
2977
3252
53
1597
3850
52
2811
1522
1143
1995
5673
1894
2848
7766
7774
------------------------
>>> Current Step 24
------------------------
{'choice': [], 'play_card': [[<Move.play_card: 4>, 0, 0, None], [<Move.play_card: 4>, 0, 1, None], [<Move.play_card: 4>, 0, 2, None], [<Move.play_card: 4>, 0, 3, None], [<Move.play_card: 4>, 0, 4, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 5, 0, None], [<Move.play_card: 4>, 5, 1, None]], 'heropower': [[<Move.hero_power: 1>, None, None]], 'minion_attack': [[<Move.minion_attack: 2>, 0, 0], [<Move.minion_attack: 2>, 0, 1], [<Move.minion_attack: 2>, 1, 0], [<

[fireplace.entity]: Player1 begins turn 9
[fireplace.entity]: Player1 is now at 4 mana crystals
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Thrall')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player1', hero=<Hero ('Thrall')>)]
[fireplace.entity]: Player1 draws <Minion ('Stranglethorn Tiger')>
[fireplace.card]: <Minion ('Stranglethorn Tiger')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player1 plays <Spell ('Lava Shock')> (target=<Minion ('Nerubian')>, index=None)
[fireplace.entity]: Player1 pays 2 mana
[fireplace.card]: <Spell ('Lava Shock')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Spell ('Lava Shock')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=<Minion ('Nerubian')>)> targeting [<Spell ('Lava Shock')>]
[fireplace.actions]: Activating <Spell ('Lava Shock')> action targeting <Minion ('Nerubian')>
[fireplace.actions]: <Spell

{'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 2, 0, None], [<Move.play_card: 4>, 2, 1, None], [<Move.play_card: 4>, 2, 2, None], [<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.play_card: 4>, 5, 0, None], [<Move.play_card: 4>, 5, 1, None], [<Move.play_card: 4>, 5, 2, None], [<Move.play_card: 4>, 5, 3, None], [<Move.play_card: 4>, 5, 4, None]], 'heropower': [[<Move.hero_power: 1>, None, None]], 'minion_attack': [[<Move.minion_attack: 2>, 0, 0], [<Move.minion_attack: 2>, 0, 1], [<Move.minion_attack: 2>, 0, 2], [<Move.minion_attack: 2>, 0, 3]], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}
[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 2, 0, None], [<Move.play_card: 4>, 2, 1, None], [<Move.play_card: 4>, 2, 2, None], [<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.play_card: 4>, 5

[fireplace.actions]: <Minion ('Jade Swarmer')> triggering <TargetedAction: Hit(<Hit.AMOUNT>=1)> targeting [<Hero ('Thrall')>]
[fireplace.actions]: <Minion ('Jade Swarmer')> triggering <TargetedAction: Predamage(<Predamage.AMOUNT>=1)> targeting [<Hero ('Thrall')>]
[fireplace.actions]: <Minion ('Jade Swarmer')> triggering <TargetedAction: Damage()> targeting [<Hero ('Thrall')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player2 plays <Minion ('Aya Blackpaw')> (target=None, index=None)
[fireplace.entity]: Player2 pays 6 mana
[fireplace.card]: <Minion ('Aya Blackpaw')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Aya Blackpaw')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Aya Blackpaw')>]
[fireplace.actions]: Activating <Minion ('Aya Blackpaw')> action targeting None
[fireplace.actions]: <Minion ('Aya Blackpaw')> triggering <TargetedAction: SummonJadeGolem()> targeting [

2848
1136
2977
3252
1597
3850
52
53
2811
1522
7982
1995
5673
1894
2848
7766
1908
7774
------------------------
>>> Current Step 29
------------------------
{'choice': [], 'play_card': [[<Move.play_card: 4>, 0, 0, None], [<Move.play_card: 4>, 0, 1, None], [<Move.play_card: 4>, 0, 2, None], [<Move.play_card: 4>, 0, 3, None], [<Move.play_card: 4>, 0, 4, None], [<Move.play_card: 4>, 0, 5, None], [<Move.play_card: 4>, 0, 6, None], [<Move.play_card: 4>, 1, None, None], [<Move.play_card: 4>, 2, None, None], [<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, 0, None], [<Move.play_card: 4>, 4, 1, None], [<Move.play_card: 4>, 4, 2, None], [<Move.play_card: 4>, 4, 3, None], [<Move.play_card: 4>, 5, None, None], [<Move.play_card: 4>, 6, None, None]], 'heropower': [[<Move.hero_power: 1>, None, None]], 'minion_attack': [[<Move.minion_attack: 2>, 1, 0], [<Move.minion_attack: 2>, 1, 1], [<Move.minion_attack: 2>, 1, 2]], 'hero_attack': [[<Move.hero_attack: 3>, None, 0], [<Move.hero_attack: 

In [7]:
## Calcular estadísticas
env.display_stats()

total_time = end-start

if(env.curr_episode > 0):
    avg_time = (end-start) / env.curr_episode
    avg_steps = steps / env.curr_episode
else:
    avg_time = (end-start) 
    avg_steps = steps 


if (env.wins + env.losses > 0):
    win_rate = env.wins / (env.wins + env.losses)
else:
    win_rate = 0

print("Avg steps per game: " + str(avg_steps))
print("Total running time: {:.2f} and running time per game: {:.2f}".format(total_time, avg_time) )

## Crear ficheros
file_name = str(env.action_type) + "__" + str(env.reward_mode) + "__" + str(steps) + "__" + str(run)

# METRICS
data_file = open('./metrics/' + file_name + '.txt', 'w')
data_file.write("Games, AvgStepsGame, TotalTime, AvgTimeGame, TotalReward, WinRate \n")
data_file.write("{},{:.2f},{:.2f},{:.2f},{:.2f},{:.2f}".format(env.curr_episode, avg_steps, total_time, avg_time, env.total_reward, win_rate))
data_file.close()

# REWARD DATA
data_file = open('./data/' + file_name + '_reward.csv', 'w')
data_file.write("Games, Reward \n")
for key in env.reward_dict:
    data_file.write("{},{:.2f} \n".format(key, env.reward_dict[key]))
data_file.close()

# Train Games
data_file = open("./data/" + file_name + '_games.csv', 'w')
for l in env.games_outcome:
    data_file.write(str(l) + "\n")
data_file.close()

model.save("./dqn_models/" + file_name + "_dqn")
env.close()


-----------STATS------------
Number of Games: 14
Number of Wins: 0
Number of Losses: 14
Number of Ties: 0
Number of Errors: 14
Number of Win Rate: 0.0
Total reward is: -14.919999999999979
----------------------------
Avg steps per game: 71.42857142857143
Total running time: 73.54 and running time per game: 5.25
